In [1]:
# wget https://www.gutenberg.org/cache/epub/70593/pg70593.txt -O book.txt

with open("book.txt","r",encoding="utf-8") as file_obj:
    text = file_obj.read()
    text = text.lower()
    sentence = text.split("\n")
sentence

['\ufeffthe project gutenberg ebook of elizabeth montagu, the queen of the',
 'bluestockings, volumes i and ii, by emily jane climenson',
 '',
 'this ebook is for the use of anyone anywhere in the united states and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever. you may copy it, give it away or re-use it under the terms',
 'of the project gutenberg license included with this ebook or online at',
 'www.gutenberg.org. if you are not located in the united states, you',
 'will have to check the laws of the country where you are located before',
 'using this ebook.',
 '',
 'title: elizabeth montagu, the queen of the bluestockings, volumes i and',
 '       ii',
 '       her correspondence from 1720 to 1761',
 '',
 'authors: emily jane climenson',
 '         elizabeth montagu',
 '',
 'release date: april 19, 2023 [ebook #70593]',
 '',
 'language: english',
 '',
 'produced by: fay dunn and the online distributed proofreading team at',
 '            

In [2]:
import tensorflow as tf
import keras
from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences


In [3]:
tokenizer = Tokenizer(oov_token="<UNK>")
toekn = tokenizer.fit_on_texts(sentence)
vocab_size = len(tokenizer.word_index) + 1
vocab_size

15617

In [4]:
sequences = tokenizer.texts_to_sequences(sentence)
sequences


[[8900, 284, 628, 1714, 3, 113, 36, 2, 482, 3, 2],
 [4158, 1045, 7, 5, 31, 35, 2325, 1847, 2840],
 [],
 [34, 1714, 9, 13, 2, 314, 3, 2559, 4159, 8, 2, 1176, 425, 5],
 [75, 137, 875, 3, 2, 258, 17, 81, 848, 5, 15, 444, 81, 6294],
 [4160, 19, 78, 573, 14, 146, 14, 397, 58, 2560, 314, 14, 264, 2, 652],
 [3, 2, 284, 628, 1046, 4161, 15, 34, 1714, 58, 3604, 17],
 [2139, 628, 2326, 67, 19, 39, 26, 2841, 8, 2, 1176, 425, 19],
 [38, 24, 4, 4162, 2, 1047, 3, 2, 176, 107, 19, 39, 2841, 138],
 [2327, 34, 1714],
 [],
 [1986, 113, 36, 2, 482, 3, 2, 4158, 1045, 7, 5],
 [31],
 [16, 653, 29, 1715, 4, 531],
 [],
 [4163, 2325, 1847, 2840],
 [113, 36],
 [],
 [4164, 849, 391, 498, 8901, 1714, 8902],
 [],
 [1716, 532],
 [],
 [1848, 35, 8903, 8904, 5, 2, 3604, 3605, 8905, 6295, 17],
 [8906, 2139, 8907, 8908, 34, 5029, 10, 1848, 29, 6296],
 [5030, 76, 5031, 35, 2, 8909, 1605],
 [],
 [3606, 3, 2, 284, 628, 1714, 113, 36, 2, 482],
 [3, 2, 4158, 1045, 7, 5, 31],
 [],
 [],
 [],
 [],
 [],
 [113, 36],
 [],
 [2, 48

In [5]:
input_sequence = []
for sequence in sequences:
    for i in range(1,len(sequence)):
        n_gram_sequence = sequence[:i+1]
        input_sequence.append(n_gram_sequence)
    


In [6]:
max_sequence_len = max([len(i) for i in input_sequence])
max_sequence_len

19

In [7]:
from keras.utils import pad_sequences
padded_sequences = pad_sequences(input_sequence,maxlen = max_sequence_len)
print(padded_sequences)

[[   0    0    0 ...    0 8900  284]
 [   0    0    0 ... 8900  284  628]
 [   0    0    0 ...  284  628 1714]
 ...
 [   0    0    0 ...    4  218  105]
 [   0    0    0 ...  218  105  197]
 [   0    0    0 ...  105  197 3167]]


In [8]:
import numpy as np
padded_sequences = np.array(padded_sequences)

x = padded_sequences[:,:-1]
labels= padded_sequences[:,-1]

In [9]:
y = keras.utils.to_categorical(labels,num_classes=vocab_size)

In [10]:
y.shape

(191859, 15617)

In [11]:
from keras.models import Sequential
from keras.layers import Embedding,Bidirectional,LSTM,Dense
from keras.optimizers import Adam

In [12]:
model = Sequential()
model.add(Embedding(vocab_size,100,input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(256)))
model.add(Dense(vocab_size,activation="softmax"))
adam= Adam(learning_rate=0.01)
model.compile(optimizer=adam,loss="categorical_crossentropy",metrics=["acc"])
model.summary()

# from keras.models import Sequential
# from keras.layers import Dense, LSTM, Embedding

# # Define the model architecture
# # model = Sequential()
# # model.add(Embedding(input_dim=vocab_size, output_dim=100, input_length=max_sequence_len-1))
# # model.add(LSTM(units=256, return_sequences=True))
# # model.add(LSTM(units=256))
# # model.add(Dense(units=vocab_size, activation='softmax'))
# # adam = Adam(learning_rate=0.01)
# # Compile the model
# model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
# model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 18, 100)           1561700   
                                                                 
 bidirectional (Bidirectiona  (None, 512)              731136    
 l)                                                              
                                                                 
 dense (Dense)               (None, 15617)             8011521   
                                                                 
Total params: 10,304,357
Trainable params: 10,304,357
Non-trainable params: 0
_________________________________________________________________


In [13]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor = "acc",patience=2, min_delta=0.01)

In [15]:
model.fit(x, y, epochs=10, verbose=1, batch_size=200, callbacks=[es])

Epoch 1/10
960/960 [==============================] - 314s 327ms/step - loss: 6.2318 - acc: 0.1139
Epoch 2/10
960/960 [==============================] - 307s 320ms/step - loss: 5.3836 - acc: 0.1505
Epoch 3/10
960/960 [==============================] - 290s 302ms/step - loss: 4.8232 - acc: 0.1747
Epoch 4/10
960/960 [==============================] - 284s 296ms/step - loss: 4.3265 - acc: 0.2030
Epoch 5/10
960/960 [==============================] - 309s 322ms/step - loss: 3.9075 - acc: 0.2370
Epoch 6/10
960/960 [==============================] - 310s 323ms/step - loss: 3.5764 - acc: 0.2701
Epoch 7/10
960/960 [==============================] - 751s 783ms/step - loss: 3.3223 - acc: 0.3017
Epoch 8/10
960/960 [==============================] - 297s 309ms/step - loss: 3.1289 - acc: 0.3273
Epoch 9/10
960/960 [==============================] - 418s 436ms/step - loss: 2.9750 - acc: 0.3498
Epoch 10/10
960/960 [==============================] - 315s 328ms/step - loss: 2.8588 - acc: 0.3660


In [16]:
#Time to become storyteller!
seed_text = "show me the real path to"          
next_words = 100
  
for _ in range(next_words):
  sequence = tokenizer.texts_to_sequences([seed_text])
  padded = pad_sequences(sequence, maxlen=max_sequence_len-1)
#   predicted = model.predict_(padded, verbose=0)
  predicted_probabilities = model.predict(padded, verbose=0)
  predicted_class = np.argmax(predicted_probabilities, axis=-1)
  output_word = ''
  for word, index in tokenizer.word_index.items():
    if index == predicted_class:
      output_word = word
      break
  seed_text += ' ' + output_word
print(seed_text)

show me the real path to the nation i have seen the establishment of its spiritual monarchy i have been a witness of it in the house and in the next letter from the duchess of portland at sandleford mrs montagu writes to the duchess on august 17 “middleton will be one who were destined to go to bed late at the heighth i am glad you have been in surrounded his affairs ” she is too near the same year before she was at the end of the letter remarks “that mankind were in the evening i have been a very pretty but a very


In [18]:
model.save('server/lstm_model.h5')

In [17]:
import pickle

# Save the fitted Tokenizer using pickle
with open('server/tokenizer.pkl', 'wb') as token_file:
    pickle.dump(tokenizer, token_file)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

# define the input text
input_text = "Hello world!"

# create a tokenizer instance
tokenizer = Tokenizer()

# fit the tokenizer on the input text
tokenizer.fit_on_texts([input_text])

# convert the input text to sequence of integers
encoded_text = tokenizer.texts_to_sequences([input_text])

# print the encoded text
print(encoded_text)

: 